In [1]:
import re
import time
import urllib
from bs4 import BeautifulSoup
import html
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import urllib
import urllib3

In [ ]:
# 여기에 전체 코드를 정리해서 입력하세요.
client_id = ''
client_secret = ''
base_url = 'https://openapi.naver.com/v1/search/news.json'
query = '재생에너지'
encQuery = urllib.parse.quote(query)
n_display = 10
start = 1
sort = 'sim'
url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'

my_request = urllib.request.Request(url)
my_request.add_header("X-Naver-Client-Id",client_id)
my_request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(my_request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
#     print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)
search_results = response_body.decode('utf-8')
search_results = eval(search_results)
all_results = dict()
i = 0
for item in search_results['items']:
    link =  html.unescape(item['link']).replace('\\','')
    pubdate = item['pubDate']
    if 'n.news.naver.com' in link:
        all_results[i] = dict()
        all_results[i]['link'] = link
        all_results[i]['pubdate'] = pubdate
        i += 1
from selenium import webdriver
driver = webdriver.Chrome()

# 페이지 로드를 위해 기다리는 시간
driver.implicitly_wait(3)

for j in range(len(all_results)):
    
    # scraping하려는 웹페이지 주소를 get()에 전달
    driver.get(all_results[j]['link'])
    time.sleep(1)
    # 제목 추출하기
    title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
    title = title[0].text
        
    # 본문 추출하기
    body = driver.find_elements(By.ID, 'newsct_article')
    # body = driver.find_elements(By.ID, 'articeBody') # 가끔 ID가 이렇게 생긴 녀석도 존재합니다...
    body = body[0].text.replace('\n','')
    all_results[j]['link'] = link
    all_results[j]['title'] = title
    all_results[j]['content'] = body
df_all_comments = pd.DataFrame(all_results).T

In [ ]:
import requests
for cnt in range(10):
    client_id = ''
    client_secret = ''
    base_url = 'https://openapi.naver.com/v1/search/news.json'
    query = '지구온난화'
    encQuery = urllib.parse.quote(query)
    # 한 번에 표시할 검색 결과 개수(기본 값: 10, 최댓 값: 100)
    n_display = 100
    # 검색 시작 위치(기본 값: 1, 최댓 값: 1000)
    start = 1 + 100*cnt
    # 검색 결과 정렬 방법(sim: 정확도 순, date: 날짜 순)
    sort = 'sim'
    url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'

    my_request = urllib.request.Request(url)
    my_request.add_header("X-Naver-Client-Id",client_id)
    my_request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(my_request)
    rescode = response.getcode()
    #정상적으로 연결 됐으면 안에 내용을 읽고 아니면 Error code를 출력합니다.
    if(rescode==200):
        response_body = response.read()
    #     print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
    search_results = response_body.decode('utf-8')
    #읽은 문자열들을 dictionary 형태로 반환합니다.
    search_results = eval(search_results)
    all_results = dict()
    i = 0
    for item in search_results['items']:
        link =  html.unescape(item['link']).replace('\\','')
        pubdate = item['pubDate']
        # 해당 문자열이 link에 있을 때를 기준으로 데이터들을 분리합니다.(보다 편한 크롤링을 위해)
        if 'n.news.naver.com' in link:
            all_results[i] = dict()
            all_results[i]['link'] = link
            all_results[i]['pubdate'] = pubdate
            i += 1
    from selenium import webdriver
    driver = webdriver.Chrome()

    # 페이지 로드를 위해 기다리는 시간
    driver.implicitly_wait(3)

    for j in range(len(all_results)):
        
        # scraping하려는 웹페이지 주소를 get()에 전달
        driver.get(all_results[j]['link'])
        time.sleep(1)
        #제목과 본문을 추출합니다.
        title = driver.find_elements(By.CLASS_NAME, 'media_end_head_headline')
        body = driver.find_elements(By.ID, 'newsct_article')
        #만약 제목과 본문에 내용이 없거나 누락되었다면(Index Error) 그냥 pass 하고
        #HTTPS 접속이 어려워져 오류가 난다면(HTTP Error) pass 합니다.
        try:
            title = title[0].text
            body = body[0].text.replace('\n','')
        except IndexError:
            pass
        except requests.exceptions.HTTPError:
            pass
        all_results[j]['link'] = link
        all_results[j]['title'] = title
        all_results[j]['content'] = body
    df_all_comments_1 = pd.DataFrame(all_results).T
    df_all_comments = pd.concat([df_all_comments,df_all_comments_1]).reset_index(drop=True)
# 여러 번의 크롤링을 통해 전체 데이터 프레임이 만들어진다면 'content' 값을 기준으로 중복을 제거합니다.
df_dust = df_all_comments.drop_duplicates(subset = ['content'],keep = 'first', inplace= False).reset_index(drop = True)
idx_lst = []
# 또한 'content' 값이 누락되거나 내용이 없는 경우(즉, None값) 해당 index를 제거합니다.
for idx,i in enumerate(df_dust['content']):
    if type(i) == list:
        idx_lst.append(idx)
df_dust.drop(index = idx_lst, inplace= True)
# 마지막으로 결측치를 제거합니다.
df_dust.dropna(inplace=True)
#이 후 전처리가 완료된 csv 파일을 저장합니다.
df_dust.to_csv('지구온난화_.csv', encoding = 'utf-8', index = False)

In [ ]:
df_all_comments.head(4)

In [ ]:
df_dust = df_all_comments.drop_duplicates(subset = ['content'],keep = 'first', inplace= False).reset_index(drop = True)
idx_lst = []
for idx,i in enumerate(df_dust['content']):
    if type(i) == list:
        idx_lst.append(idx)
df_dust.drop(index = idx_lst, inplace= True)
df_dust.dropna(inplace=True)
df_dust

In [ ]:
df_dust.to_csv('대체에너지.csv', encoding = 'utf-8', index = False)

In [ ]:
'''a = pd.read_csv("대기 오염 미세먼지_news.csv",encoding = 'utf-8')
a1 = pd.read_csv("미세먼지.csv",encoding = 'utf-8')
a2 = pd.read_csv("미세먼지_계절.csv",encoding = 'utf-8')
a3 = pd.read_csv("미세먼지_농도_news.csv",encoding = 'utf-8')
a4 = pd.read_csv("미세먼지_문제.csv",encoding = 'utf-8')
a5 = pd.read_csv("미세먼지_배출_사업장_news.csv",encoding = 'utf-8')
a6 = pd.read_csv("미세먼지_이유.csv",encoding = 'utf-8')
a7 = pd.read_csv("미세먼지_주요지역.csv",encoding = 'utf-8')
a8 = pd.read_csv("미세먼지_키워드.csv",encoding = 'utf-8')
a9 = pd.read_csv("미세먼지_해결_news.csv",encoding = 'utf-8')
a10 = pd.read_csv("미세먼지관련_정책_news.csv",encoding = 'utf-8')
a11 = pd.read_csv("초미세먼지_news.csv",encoding = 'utf-8')
a12 = pd.read_csv("초미세먼지_문제_news.csv",encoding = 'utf-8')
a13 = pd.read_csv("초미세먼지_원인_news.csv",encoding = 'utf-8')
a14 = pd.read_csv("초미세먼지_해결_news.csv",encoding = 'utf-8')
a15 = pd.read_csv("초미세먼지_해결_wordcloud_파생_.csv",encoding = 'utf-8')
a16 = pd.read_csv("환경문제_.csv",encoding = 'utf-8')
a17 = pd.read_csv("지구온난화_.csv", encoding = 'utf-8')
a18 = pd.read_csv("대기오염.csv", encoding = 'utf-8')
a19 = pd.read_csv("토양오염.csv", encoding = 'utf-8')
a20 = pd.read_csv("대체에너지.csv", encoding= 'utf-8')'''

In [ ]:
# 주의사항 
# ex)미세먼지 keyword 
# => 미세 먼지, 미세먼지 (이렇게 띄워쓰기 등을 활용하여 최대한 데이터를 많이 뽑게 진행)
# 이로 인해 duplicate의 중요성이 더 강조

In [ ]:
'''환경 = pd.concat([a,a17,a1,a11,a8], axis = 0).reset_index(drop=True)
환경문제 = pd.concat([a3,a4,a12,a16], axis = 0).reset_index(drop=True)
환경해결 = pd.concat([a9,a14,a20])
환경원인 = pd.concat([a6,a13], axis=0).reset_index(drop = True)
wordcloud_파생_정책관련 = pd.concat([a2,a5,a7,a10],axis = 0).reset_index(drop=True)
wordcloud_파생_오염관련 = pd.concat([a18,a19,a15],axis = 0).reset_index(drop=True)'''

In [ ]:
'''
wordcloud_파생_정책관련.dropna(subset=['content'], how='any', axis=0, inplace = True)
wordcloud_파생_정책관련 = wordcloud_파생_오염관련.drop_duplicates(keep = 'first', subset= ['content']).reset_index(drop = True)
wordcloud_파생_정책관련.drop(columns = ['Unnamed: 0','link','pubdate'], inplace = True)

wordcloud_파생_오염관련.dropna(subset=['content'], how='any', axis=0, inplace = True)
wordcloud_파생_오염관련 = wordcloud_파생_오염관련.drop_duplicates(keep = 'first', subset= ['content']).reset_index(drop = True)
wordcloud_파생_오염관련.drop(columns = ['Unnamed: 0','link','pubdate'], inplace = True);wordcloud_파생_오염관련'''

In [ ]:
'''
환경.to_csv('환경.csv', encoding = 'utf-8', index = False)
환경문제.to_csv('환경문제.csv', encoding = 'utf-8', index = False)
환경해결.to_csv('환경해결.csv', encoding = 'utf-8', index = False)
환경원인.to_csv('환경원인.csv', encoding = 'utf-8', index = False)
wordcloud_파생_정책관련.to_csv('wordcloud_파생_정책관련.csv', encoding = 'utf-8', index = False)
wordcloud_파생_오염관련.to_csv('wordcloud_파생_오염관련.csv', encoding = 'utf-8', index = False)'''